In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [16]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [17]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [18]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [19]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 4.45 s, sys: 31.5 ms, total: 4.48 s
Wall time: 4.49 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1017041, 937254, 851644, 2662416, 7147659]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1219382, 75637, 9270383, 987229, 7166780]"


Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу
Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [20]:
#функция для получения весов товаров в зависимости от объёма продаж в денежном эквиваленте
def get_items_weights(df):
    total_sales=df['sales_value'].sum()
    items_weights=df.groupby('item_id').agg({'sales_value':'sum'}).reset_index().rename(columns={'sales_value':'weight'})
    items_weights['weight']=items_weights['weight'].apply(lambda x: x/total_sales)
    return items_weights

In [21]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    weights=np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False) #используем параметр p метода choice в который передадим веса товаров
    
    
    return recs.tolist()

In [22]:
%%time

items_weights=get_items_weights(data_train)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 4.15 s, sys: 427 ms, total: 4.58 s
Wall time: 4.63 s


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1017041, 937254, 851644, 2662416, 7147659]","[6534178, 981165, 1002453, 855557, 819594]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1219382, 75637, 9270383, 987229, 7166780]","[954284, 9527261, 849330, 8116306, 856818]"


Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [23]:
import os
os.makedirs('../predictions', exist_ok=True)
result.to_csv('../predictions/predictions_hw.csv', index=False)

In [24]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

In [25]:
score = {}
for name_col in result.columns[1:]:
    score[name_col]= round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0007:random_recommendation
0.0229:weighted_random_recommendation


In [26]:
list(score.keys())

['actual', 'random_recommendation', 'weighted_random_recommendation']

In [27]:
precision_at_k_score = {"precision_at_k":pd.Series(list(score.values()),
                                        index=list(score.keys()))}
df_score = pd.DataFrame(precision_at_k_score)
df_score = df_score.sort_values('precision_at_k', ascending=False)
df_score

,precision_at_k
actual,1.0000
weighted_random_recommendation,0.0229
random_recommendation,0.0007


In [28]:
score = {}
for name_col in result.columns[1:]:
    score[name_col]= round(result.apply(lambda row: recall_at_k(row[name_col], row['actual']), axis=1).mean(),4)
    print(f"{round(result.apply(lambda row: recall_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

nan:actual
nan:random_recommendation
nan:weighted_random_recommendation


In [29]:
recall_at_k_score = {"recall_at_k":pd.Series(list(score.values()),
                                        index=list(score.keys()))}
df_score = df_score.join(pd.DataFrame(recall_at_k_score))
df_score = df_score.sort_values('recall_at_k', ascending=False)
df_score

,precision_at_k,recall_at_k
actual,1.0000,NaN
weighted_random_recommendation,0.0229,NaN
random_recommendation,0.0007,NaN
